![](https://raw.githubusercontent.com/BenitaDiop/students/master/bin/Cables.png)

# LOAD PACKAGES 

In [1]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1590662666009_0004,pyspark,idle,Link,Link,
4,application_1590662666009_0005,pyspark,idle,Link,Link,


In [2]:
sc.install_pypi_package("matplotlib==3.2.1")
sc.install_pypi_package("pandas==1.0.3")
sc.install_pypi_package("seaborn==0.10.0")
sc.install_pypi_package("numpy==1.18.4")
sc.install_pypi_package("pyspark==2.4.5")
sc.install_pypi_package("ipython==7.14.0")
sc.install_pypi_package("subprocess.run==0.0.8")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1590662666009_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached matplotlib-3.2.1-cp36-cp36m-manylinux1_x86_64.whl (12.4 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)
  Using cached kiwisolver-1.2.0-cp36-cp36m-manylinux1_x86_64.whl (88 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)

  Using cached pandas-1.0.3-cp36-cp36m-manylinux1_x86_64.whl (10.0 MB)

  Using cached seaborn-0.10.0-py3-none-any.whl (215 kB)
  Using cached scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl (26.1 MB)

  Using cached numpy-1.18.4-cp36-cp36m-manylinux1_x86_64.whl (20.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.14.5
    Not uninstalling numpy at /usr/local/lib64/python3.6/site-packages, outside environment /tmp/1590671609804-0
    Can't uninstall 'numpy'. No files were found to uninstall.

Processing /mnt/var/lib/livy/.cache/pip/wheels/84/30/e3/c51c5cd0229631e662d29d7b578a3e5949a4c8db033ffb70aa/pyspark-2.4.5-py2.py3-none-any.whl


In [3]:
from IPython import display 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd
from pyspark import *
import pyspark.sql.functions as f
from pyspark.sql import functions as f
from pyspark.sql.functions import rank, sum, col 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql import Window 
window = Window.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

![](https://raw.githubusercontent.com/BenitaDiop/students/master/bin/Cables.png)

![](https://raw.githubusercontent.com/BenitaDiop/students/master/bin/Cables.png)

# DATA EXPLORATION 

In [5]:
usr = spark.read.json("s3://my-little-pony/yelp/yelp_academic_dataset_user.json")
view = spark.read.load('s3://my-little-pony/yelp/reviews.json', format='json')
business = spark.read.json("s3://my-little-pony/yelp/yelp_academic_dataset_business.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:

usr.createOrReplaceTempView("usr")
query = """
SELECT 
name,
user_id as ID,
explode(split(elite, ',')),
elite
FROM usr"""
usr1 = spark.sql(query).withColumnRenamed("col", "year").drop("elite")



usr.createOrReplaceTempView("csv")
query = """
SELECT *, 
explode(split(elite, ',')),
elite
FROM csv"""
csv =spark.sql(query)



usr = usr1.groupBy("name", "ID").count().withColumnRenamed('count', 'YRSelite').sort("YRSelite", ascending=False)


usr = usr.withColumn('eliteSTAT', 
              f.when(usr.YRSelite > 1, 1).otherwise(0))




t = usr.select(["YRSelite", "eliteSTAT"])\
            .groupBy('eliteSTAT')\
            .agg(f.count('YRSelite')\
                 .alias('Years_Num'), 
                 f.mean('YRSelite')\
                 .alias('Years_Avg'),
                 f.min('YRSelite')\
                 .alias('Years_Min'),
                 f.max('YRSelite')\
                 .alias('Years_Max'))\
.withColumn('total', sum(col('Years_Num')).over(window))\
.withColumn('Percent %', f.format_string("%5.0f%%\n", col('Years_Num')*100/col('total')))
tab = t.drop('Percent %').withColumn('Percent', col('Years_Num')*100/col('total'))
t.drop(col('total'))


tab = tab.toPandas()




sub = csv.select("user_id", 
                 "fans", 
                 "cool", 
                 "useful", 
                 "average_stars",
                "compliment_cool",
                "compliment_funny", 
                "compliment_hot",
                "compliment_more",
                "compliment_note",
                "compliment_plain", 
                "compliment_photos",
                "compliment_profile")
usr.createOrReplaceTempView("us")
us = usr.join(sub, usr.ID == sub.user_id, "full")
us = us.withColumnRenamed('compliment_cool', 'cc')\
       .withColumnRenamed('compliment_funny', 'cf')\
       .withColumnRenamed('compliment_profile', 'cpr')\
       .withColumnRenamed('compliment_note', 'cn')\
       .withColumnRenamed('compliment_more', 'cm')\
       .withColumnRenamed('compliment_photos', 'cph')\
       .withColumnRenamed('compliment_plain', 'cpl')\
       .withColumnRenamed('compliment_hot', 'ch').drop("user_id")

us.columns



view.createOrReplaceTempView("tmp2")
query = """
SELECT
user_id,
stars as ReviewRating,
business_id,
review_id,
text, 
date 
FROM tmp2
"""



tmp2 = spark.sql(query)
tmp2.show()
expo = tmp2.join(us, us.ID == tmp2.user_id, "full")


expo.createOrReplaceTempView("df")
query = """
(SELECT * FROM df)"""

df = spark.sql(query)
df = df.distinct()
df = df.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------------+--------------------+--------------------+-------------------+
|             user_id|ReviewRating|         business_id|           review_id|                text|               date|
+--------------------+------------+--------------------+--------------------+--------------------+-------------------+
|OwjRMXRC0KyPrIlcj...|         2.0|-MhfebM0QIsKt87iD...|xQY8N_XvtGbearJ5X...|As someone who ha...|2015-04-15 05:21:16|
|nIJD_7ZXHq-FX8byP...|         1.0|lbrU8StCq3yDfr-QM...|UmFMZ8PyXZTY2Qcwz...|I am actually hor...|2013-12-07 03:16:52|
|V34qejxNsCbcgD8C0...|         5.0|HQl28KMwrEKHqhFrr...|LG2ZaYiOgpr2DK_90...|I love Deagan's. ...|2015-12-05 03:18:11|
|ofKDkJKXSKZXu5xJN...|         1.0|5JxlZaqCnk1MnbgRi...|i6g_oA9Yf9Y31qt0w...|Dismal, lukewarm,...|2011-05-27 05:30:52|
|UgMW8bLE0QMJDCkQ1...|         4.0|IS4cv902ykd8wj1TR...|6TdNDKywdbjoTkize...|Oh happy day, fin...|2017-01-14 21:56:57|
|5vD2kmE25YBrbayKh...|         5.0|nlxHRv1zXGT0c


![](https://raw.githubusercontent.com/BenitaDiop/students/master/bin/Cables.png)

# 80/20 SPLIT 

In [7]:
train_df,test_df=df.randomSplit([0.9,0.1])
val, tdf = test_df.randomSplit([0.99, .01])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
print("""
Number of Rows in FULL Data {} \n

Number of Rows in Test Data {} \n 

Number of rows in Sub Test Data {} 
""".format(df.count(), train_df.count(), tdf.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Number of Rows in FULL Data 8021122 


Number of Rows in Test Data 7219284 
 

Number of rows in Sub Test Data 7887

In [ ]:
tdf = tdf.toPandas()
tdf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


![](https://raw.githubusercontent.com/BenitaDiop/students/master/bin/Cables.png)

# REVIEW TEXT DATA ANALYSIS 

In [ ]:
from pyspark.ml.feature import Tokenizer

tokenization=Tokenizer(inputCol='text',outputCol='tokens')

tokenized_df=tokenization.transform(expo)

tokenized_df.select("text", "tokens").show(5, True, True)

In [ ]:
from pyspark.ml.feature import StopWordsRemover

stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

refined_df=stopword_removal.transform(tokenized_df)

refined_df.select(['user_id','tokens','refined_tokens']).show(2, True, True)

In [ ]:
from pyspark.ml.feature import CountVectorizer

count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

cv_df=count_vec.fit(refined_df).transform(refined_df)

cv_df.select(['user_id','refined_tokens','features']).show(4,False)

In [ ]:
count_vec.fit(refined_df).vocabulary

from pyspark.ml.feature import HashingTF,IDF

hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features')

hashing_df=hashing_vec.transform(refined_df)

hashing_df.select(['user_id','refined_tokens','tf_features']).show(4,False)

In [ ]:
tf_idf_vec=IDF(inputCol='tf_features',outputCol='tf_idf_features')

tf_idf_df=tf_idf_vec.fit(hashing_df).transform(hashing_df)

tf_idf_df.select(['user_id','tf_idf_features']).show(4,False)

# CLASSIFICATION 

In [ ]:
text_df=spark.read.csv('Movie_reviews.csv',inferSchema=True,header=True,sep=',')
text_df.printSchema()
text_df.count()

In [ ]:
from pyspark.sql.functions import rand 
text_df.orderBy(rand()).show(10,False)
text_df=text_df.filter(((text_df.Sentiment =='1') | (text_df.Sentiment =='0')))

text_df.count()
text_df.groupBy('Sentiment').count().show()


text_df.printSchema()
text_df = text_df.withColumn("Label", text_df.Sentiment.cast('float')).drop('Sentiment')
text_df.orderBy(rand()).show(10,False)
text_df.groupBy('label').count().show()


In [ ]:
# Add length to the dataframe
from pyspark.sql.functions import length
text_df=text_df.withColumn('length',length(text_df['Review']))

text_df.orderBy(rand()).show(10,False)
text_df.groupBy('Label').agg({'Length':'mean'}).show()

In [ ]:
# Data Cleaning
tokenization=Tokenizer(inputCol='Review',outputCol='tokens')
tokenized_df=tokenization.transform(text_df)

tokenized_df.show()

stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')
refined_text_df=stopword_removal.transform(tokenized_df)

refined_text_df.show()

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [ ]:
len_udf = udf(lambda s: len(s), IntegerType())

refined_text_df = refined_text_df.withColumn("token_count", len_udf(col('refined_tokens')))

refined_text_df.orderBy(rand()).show(10)
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')
cv_text_df=count_vec.fit(refined_text_df).transform(refined_text_df)
cv_text_df.select(['refined_tokens','token_count','features','Label']).show(10)

In [ ]:
#select data for building model
model_text_df=cv_text_df.select(['features','token_count','Label'])

In [ ]:
from pyspark.ml.feature import VectorAssembler

df_assembler = VectorAssembler(inputCols=['features','token_count'],outputCol='features_vec')
model_text_df = df_assembler.transform(model_text_df)
model_text_df.printSchema()

In [ ]:
from pyspark.ml.classification import LogisticRegression
#split the data 
training_df,test_df=model_text_df.randomSplit([0.75,0.25])
training_df.groupBy('Label').count().show()

In [ ]:
test_df.groupBy('Label').count().show()
log_reg=LogisticRegression(featuresCol='features_vec',labelCol='Label').fit(training_df)
results=log_reg.evaluate(test_df).predictions
results.show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
#confusion matrix
true_postives = results[(results.Label == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.Label == 0) & (results.prediction == 0)].count()
false_positives = results[(results.Label == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.Label == 1) & (results.prediction == 0)].count()

In [ ]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

precision = float(true_postives) / (true_postives + false_positives)
print(precision)


accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)